In [26]:
import numpy as np
import copy
import random
import os
import time
import json
import glob


In [54]:
penalty_rate = 1000
npop_list = [50, 150, 300]
ngen = 300
nseed = 5

In [13]:
def loadData(path):
    f = open(path, "r")
    data = json.load(f)

    N, m, M, d, s, e = data["N"], data["m"], data["M"], data["d"], data["s"], data["e"]
    date = np.max(e)+1
    return  N, m, M, d, s, e, date

In [24]:
def cost(N, m, M, d, date, sol):
    amount = np.zeros(date)
    penalty = 0
    for i in range(N):
        amount[sol[i]] += int(d[i])
    for i in amount:
        if (i > M):
            penalty += i-M
        elif (i > 0 and i < m):
            penalty += m-i
    amount_temp = np.delete(amount, amount == 0)
    cost = np.max(amount_temp)-np.min(amount_temp)
    return int(cost+penalty_rate*penalty)


In [18]:
def initialSolution(N, m, M, d, s, e, date, SEED):
    random.seed(SEED)
    sol = [0]*N
    amount = np.zeros(date)
    global flag
    flag = [0]*date

    # cánh đồng nào thu nhiều thì ưu tiên chọn trước
    order = np.argsort(-np.array(d))
    for i in order:
        temp1 = [j for j in range(s[i], e[i]+1) if flag[j] <= 1]
        temp2 = [j for j in range(s[i], e[i]+1)
                 if (flag[j] == 2 and amount[j] + d[i] <= M)]
        if (len(temp1) != 0):
            day = random.choice(temp1)
        elif (len(temp2) != 0):
            day = random.choice(temp2)
        else:
            day = random.randint(s[i], e[i])

        sol[i] = day
        amount[day] += d[i]
        if (amount[day] > M):
            flag[day] = 3  # đã đến giới hạn
        elif (amount[day] >= m):
            flag[day] = 2
        elif (amount[day] > 0):
            flag[day] = 1
            
    return sol


In [19]:
def sort(sol):
    return sol[-1]


In [57]:
def GA(N, m, M, d, s, e, date, npop, SEED):
    random.seed(SEED)
    # khởi tạo quần thể
    population = []
    solution_gen = []
    for i in range(npop):
        sol = [0]*N
        sol = initialSolution(N, m, M, d, s, e, date, SEED + i*10)
        sol.append(cost(N, m, M, d, date, sol))
        population.append(sol)
    population.sort(reverse=False, key=sort)  # sắp xếp cá thể theo cost

    for k in range(ngen):
        lamda = random.uniform(0, 1)  # tỉ lệ lai ghép và đột biến
        if (lamda < 0.9):
            # lai ghép
            new_generation = []
            size = int(len(population)/10)
            for _ in range(int((len(population) - size)/2)):
                # chọn cá thể lai ghép đầu tiên
                number1 = random.randint(0, int(len(population)/2))
                # chọn cá thể lai ghép thư hai
                number2 = random.randint(0, int(len(population)/2))
                while (number1 == number2):
                    number2 = random.randint(0, int(len(population)/2))
                x1 = random.randint(1, N-1)  # chọn điểm cắt lai ghép thứ 1
                x2 = random.randint(1, N-1)  # chọn điểm cắt lai ghép thứ 2

                while (x1 == x2):
                    x1 = random.randint(1, N-1)
                if (x1 > x2):
                    x1, x2 = x2, x1

                new_individual_1 = [0]*N  # cá thể con 1
                new_individual_2 = [0]*N  # cá thể con 2
                for i in range(x1):
                    new_individual_1[i] = population[number1][i]
                    new_individual_2[i] = population[number2][i]
                for i in range(x1, x2):
                    new_individual_1[i] = population[number2][i]
                    new_individual_2[i] = population[number1][i]
                for i in range(x2, N):
                    new_individual_1[i] = population[number1][i]
                    new_individual_2[i] = population[number2][i]

                new_individual_1.append(cost(N, m, M, d, date, new_individual_1))
                new_generation.append(new_individual_1)
                new_individual_2.append(cost(N, m, M, d, date, new_individual_2))
                new_generation.append(new_individual_2)

            for i in population[:size]:
                new_generation.append(i)
            population = new_generation
            # sắp xếp cá thể theo cost
            population.sort(reverse=False, key=sort)
        else:
            # đột biến
            # chọn cá thể đột biến
            number3 = random.randint(0, len(population)-1)
            x = random.randint(0, N-1)  # chọn gen đột biến
            while (s[x] == e[x]):
                x = random.randint(0, N-1)

            new_individual_3 = [0]*N  # khởi tạo cá thể đột biến
            for i in range(N):
                new_individual_3[i] = population[number3][i]

            new_individual_3[x] = random.randint(s[x], e[x])
            while (new_individual_3[x] == population[number3][x]):
                new_individual_3[x] = random.randint(s[x], e[x])
            new_individual_3 = np.append(
                new_individual_3, cost(N, m, M, d, date, new_individual_3))
            del population[-1]  # xóa cá thể kém
            population.append(new_individual_3) # thêm cá thể đột biến vào quần thể
            # sắp xếp cá thể theo cost
            population.sort(reverse=False, key=sort) 
        solution_gen.append(population[0])
        # if (population[0][-1] < 50):
        #     break
    return solution_gen


In [50]:
def logOutput_solution(outputPath, Name, time, N, d, date, SEED, solution_gen):
    sol = solution_gen[-1]
    outputPath = outputPath + ".txt"
    f = open(outputPath, "w")
    f.write("Name: "+Name+"\n")
    f.write("Time: {} \n".format(time))
    f.write("Result: {}\n".format(sol[-1]))
    f.write("Solution: ")
    for i in range(len(sol)-1):
        f.write(" {}".format(sol[i]))
    f.write("\n")
    amount = np.zeros(date)
    day = [[] for i in range(date)]
    for i in range(N):
        amount[sol[i]] += d[i]
        day[sol[i]].append(i)

    for i in range(date):
        f.write("{}:".format(i))
        for j in range(len(day[i])):
            f.write(" {}".format(day[i][j]))
        f.write(" ({})\n".format(amount[i]))


In [52]:
def logOutput_gen(outputPath, Name, SEED, solution_gen):
    sol = solution_gen[-1]
    outputPath = outputPath + "_gen.txt"
    f = open(outputPath, "w")
    f.write("File name: " + Name + "\n")
    f.write("Result: " + str(sol[-1]) + "\n")
    for i in range(1, len(solution_gen)+1):
        f.write("Gen " + str(i) + ": " + str(solution_gen[i-1][-1]) + "\n")


# Test

In [59]:
for npop in npop_list:
    print("Population: ", npop)
    if os.path.exists(f"../results/GeneticAlgorithm/{npop}_population") == False:
        os.mkdir(f"../results/GeneticAlgorithm/{npop}_population")
    for path in glob.glob("../data/data_v2/Type1Small/**.json"):
        type, name = path.split("/")[-2:]
        name_list = name[:-5].split("_")
        name_fix = "-".join(name_list[-3:])
        print("Type: ", type, " | Name: ", name_fix)
        if os.path.exists(f"../results/GeneticAlgorithm/{npop}_population/{type}") == False:
            os.mkdir(f"../results/GeneticAlgorithm/{npop}_population/{type}")
        N, m, M, d, s, e, date = loadData(f"../data/data_v2/{type}/{name}")
        for SEED in range(nseed):
            outputPath = "../results/GeneticAlgorithm/" + str(npop) + "_population/" + type + "/" + "GA(" + name_fix + ")_seed" + str(SEED)
            start = time.time()
            solution_gen = GA(N, m, M, d, s, e, date, npop, SEED)
            end = time.time()
            logOutput_solution(outputPath, name_fix, end-start, N, d, date, SEED, solution_gen)
            logOutput_gen(outputPath, name_fix, SEED, solution_gen)

Population:  50
Type:  Type1Small  | Name:  20-30-50
Type:  Type1Small  | Name:  20-20-30
Type:  Type1Small  | Name:  20-10-15
Type:  Type1Small  | Name:  50-40-75
Type:  Type1Small  | Name:  10-10-15
Type:  Type1Small  | Name:  50-50-100
Type:  Type1Small  | Name:  20-50-100
Type:  Type1Small  | Name:  10-20-30
Type:  Type1Small  | Name:  40-30-50
Type:  Type1Small  | Name:  30-50-100
Type:  Type1Small  | Name:  40-50-100
Type:  Type1Small  | Name:  10-30-50
Type:  Type1Small  | Name:  40-10-15
Type:  Type1Small  | Name:  30-40-75
Type:  Type1Small  | Name:  40-20-30
Type:  Type1Small  | Name:  30-10-15
Type:  Type1Small  | Name:  40-40-75
Type:  Type1Small  | Name:  10-50-100
Type:  Type1Small  | Name:  30-20-30
Type:  Type1Small  | Name:  30-30-50
Type:  Type1Small  | Name:  10-40-75
Type:  Type1Small  | Name:  50-20-30
Type:  Type1Small  | Name:  50-10-15
Type:  Type1Small  | Name:  20-40-75
Type:  Type1Small  | Name:  50-30-50
Population:  150
Type:  Type1Small  | Name:  20-30-50


In [60]:
npop = 150
print("Population: ", npop)
for path in glob.glob("../data/data_v2/**/**.json"):
    type, name = path.split("/")[-2:]
    name_list = name[:-5].split("_")
    name_fix = "-".join(name_list[-3:])
    print("Type: ", type, " | Name: ", name_fix)
    if os.path.exists(f"../results/GeneticAlgorithm/{npop}_population/{type}") == False:
        os.mkdir(f"../results/GeneticAlgorithm/{npop}_population/{type}")
    N, m, M, d, s, e, date = loadData(f"../data/data_v2/{type}/{name}")
    for SEED in range(nseed):
        outputPath = "../results/GeneticAlgorithm/" + str(npop) + "_population/" + type + "/" + "GA(" + name_fix + ")_seed" + str(SEED)
        start = time.time()
        solution_gen = GA(N, m, M, d, s, e, date, npop, SEED)
        end = time.time()
        logOutput_solution(outputPath, name_fix, end-start, N, d, date, SEED, solution_gen)
        logOutput_gen(outputPath, name_fix, SEED, solution_gen)

Population:  150
Type:  Type5Large  | Name:  250-30-50
Type:  Type5Large  | Name:  100-50-100
Type:  Type5Large  | Name:  500-50-100
Type:  Type5Large  | Name:  500-20-30
Type:  Type5Large  | Name:  750-50-100
Type:  Type5Large  | Name:  1000-60-125


KeyboardInterrupt: 